Inital EDA and Pre-processing

Steps left to do: hande missing values

In [16]:
#importing libraries
import pandas as pd

In [17]:
#Mount drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
#Read json sample into a dataframe
df = pd.read_json('/content/drive/MyDrive/BU_MSBA/BA820 - Unsupervised ML/random_sample_100.json')
df.head()

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,image,vote
0,5,False,"01 23, 2017",A1YTZAZZ1L32RL,0002005263,{'Format:': ' Kindle Edition'},Chuck,Ok reading,Ok,1485129600,NaN,NaN
1,5,True,"08 3, 2014",A3EHZMD24B5BQ0,0002005263,{'Format:': ' Paperback'},Sola49,Good,Five Stars,1407024000,NaN,NaN
2,5,True,"07 9, 2013",A38ZMH9LRCZL3X,0001381733,{'Format:': ' Hardcover'},A. E. Stoff,I purchased it for my new great-granddaughter....,New Book for New Grandchild,1373328000,NaN,NaN
3,5,True,"12 3, 2017",AYMEYAL8G7LJB,0001384198,{'Format:': ' Hardcover'},MARILYN,A very nice replica of the original classic.,Excellent reproduction of the timeless classic.,1512259200,NaN,NaN
4,3,False,"10 4, 2011",A39QEKNEFSJ34Q,0002005263,{'Format:': ' Unknown Binding'},hrladyship,If you're thinking of buying The Sinister Pig ...,And the hero is . . .,1317686400,NaN,NaN


In [19]:
#Read csv with metadata by asin code
meta_data = pd.read_csv('/content/drive/MyDrive/BU_MSBA/BA820 - Unsupervised ML/books_meta_short.csv')

In [21]:
#Perform a left join to add metadata information (title, price) to each book for
#each book review
merged_df = pd.merge(df, meta_data, on='asin', how='left')

In [22]:
#Remove original dataframes
import gc #garbage collector for deleted dataframes

del df
del meta_data

gc.collect

<function gc.collect(generation=2)>

In [24]:
#Check dtypes and columns
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61100 entries, 0 to 61099
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   overall         61100 non-null  int64 
 1   verified        61100 non-null  bool  
 2   reviewTime      61100 non-null  object
 3   reviewerID      61100 non-null  object
 4   asin            61100 non-null  object
 5   style           58295 non-null  object
 6   reviewerName    61098 non-null  object
 7   reviewText      61094 non-null  object
 8   summary         61091 non-null  object
 9   unixReviewTime  61100 non-null  int64 
 10  image           109 non-null    object
 11  vote            11773 non-null  object
 12  Unnamed: 0      61100 non-null  int64 
 13  title           61100 non-null  object
 14  price           52443 non-null  object
dtypes: bool(1), int64(3), object(11)
memory usage: 7.1+ MB


In [26]:
#Convert style from dictionary into standard dataframe column
merged_df['style'] = merged_df['style'].apply(lambda x: x['Format:'].strip()\
                                if pd.notnull(x) and 'Format:' in x else x)

merged_df['style'].head()

0     Kindle Edition
1          Paperback
2          Hardcover
3          Hardcover
4    Unknown Binding
Name: style, dtype: object

In [27]:
#Drop image column since it is beyond of the scope of this project
merged_df.drop(columns='image', inplace=True)

In [28]:
#Check all unique values for each column, this will help us check the nature of
#each variable and if there are more missing values listed as '?' or 'missing'

#Print unique values for each variable
print("Unique values in 'overall':", merged_df['overall'].unique(), "\n")
print("Unique values in 'verified':", merged_df['verified'].unique(), "\n")
print("Unique values in 'reviewTime':", merged_df['reviewTime'].unique(), "\n")
print("Unique values in 'reviewerID':", merged_df['reviewerID'].unique(), "\n")
print("Unique values in 'asin':", merged_df['asin'].unique(), "\n")
print("Unique values in 'style':", merged_df['style'].unique(), "\n")
print("Unique values in 'reviewerName':", merged_df['reviewerName'].unique(), "\n")
print("Unique values in 'reviewText':", merged_df['reviewText'].unique(), "\n")
print("Unique values in 'summary':", merged_df['summary'].unique(), "\n")
print("Unique values in 'unixReviewTime':", merged_df['unixReviewTime'].unique(), "\n")
print("Unique values in 'vote':", merged_df['vote'].unique(), "\n")

Unique values in 'overall': [5 3 4 2 1] 

Unique values in 'verified': [False  True] 

Unique values in 'reviewTime': ['01 23, 2017' '08 3, 2014' '07 9, 2013' ... '06 1, 2002' '10 3, 1999'
 '04 6, 2001'] 

Unique values in 'reviewerID': ['A1YTZAZZ1L32RL' 'A3EHZMD24B5BQ0' 'A38ZMH9LRCZL3X' ... 'A1ZB28DKVCEKAG'
 'A27NFSLWVBJK32' 'ADY836HK6QSYR'] 

Unique values in 'asin': ['0002005263' '0001381733' '0001384198' ... '0060197552' '0060255404'
 '0060224827'] 

Unique values in 'style': ['Kindle Edition' 'Paperback' 'Hardcover' 'Unknown Binding'
 'Mass Market Paperback' 'Board book' 'Library Binding' nan
 'Audible Audiobook' 'Audio CD' 'Audio Cassette' 'Bonded Leather'
 'School & Library Binding' 'Leather Bound' 'Vinyl' 'MP3 CD'
 'Imitation Leather' 'Spiral-bound' 'MP3 Music' 'Amazon Video' 'DVD'
 'VHS Tape' 'Loose Leaf' 'Kindle Edition with Audio/Video' 'Vinyl Bound'
 'Cards' 'Rag Book' 'Digital' 'MP3 CD Library Binding' 'Map' 'Flexibound'
 'DVD Audio' 'Poster' 'Calendar' 'DVD-ROM' 'Plastic 

In [29]:
#Drop reviewerName, reviewerID, and unixReviewTime(we already have a review time var)
merged_df.drop(columns=['reviewerName','reviewerID', 'unixReviewTime'], inplace=True)

In [31]:
#Convert reviewTime to datetime (previously an object type)
merged_df['reviewTime'] = pd.to_datetime(merged_df['reviewTime'], format='%m %d, %Y')

Note: `asin` is a product code, I left in in because maybe we can figure out a way to get more product details and names from this

In [32]:
#Check distribution of scores
print('Distribution of Book overall ratings:')
merged_df['overall'].value_counts(normalize=True)

Distribution of Book overall ratings:


5    0.628216
4    0.203748
3    0.090851
2    0.041702
1    0.035483
Name: overall, dtype: float64

In [33]:
print('Distribution of Book formats')
merged_df['style'].value_counts(normalize=True)

Distribution of Book formats


Kindle Edition                     0.484896
Paperback                          0.227944
Hardcover                          0.187049
Mass Market Paperback              0.066129
Audio CD                           0.010327
Board book                         0.009109
Audible Audiobook                  0.002316
Audio Cassette                     0.001853
Library Binding                    0.001544
Kindle Edition with Audio/Video    0.001424
Spiral-bound                       0.001132
Leather Bound                      0.000738
DVD                                0.000703
Imitation Leather                  0.000635
MP3 CD                             0.000618
Unknown Binding                    0.000480
Amazon Video                       0.000446
Hardcover-spiral                   0.000446
School & Library Binding           0.000377
Map                                0.000257
MP3 Music                          0.000206
Ring-bound                         0.000206
Plastic Comb                    